In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv("data/client_knp.csv")
df.columns = ["client", "knp"]
df.knp = df.knp.str.replace(",", " ")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
from gensim.models.word2vec import Word2Vec

word2vec = Word2Vec.load("data/w2v.model")
vocab = word2vec.wv.vocab.keys()
word_vectors = {key:word2vec.wv[key] for key in vocab}
vectors = [word2vec.wv[key] for key in vocab]

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
_ = tfidf.fit_transform(df.knp)
idfs = {key:tfidf.idf_[value] for key, value in tfidf.vocabulary_.items()}

In [8]:
def get_vector(knps):
    knps = list(filter(lambda knp: knp in word_vectors.keys() and knp in idfs.keys(), knps.split()))
    
    if not knps:
        return np.zeros(shape=vectors[0].shape[0])
    
    tf = 1.0 / len(knps)
    weights = np.array([idfs[knp] * tf for knp in knps])
    knp_vecs = np.array([word_vectors[knp] for knp in knps])
    
    return np.multiply(knp_vecs, weights.reshape(-1, 1)).sum(axis=0)

df["w2v"] = df.knp.apply(get_vector)

In [11]:
a = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]])
b = np.array([0.5, 0.5, 0.5])

a.dot(b)

array([1.5, 3. , 4.5])

In [ ]:
%%time 

def get_distance(w2v):
    subset = df.sample(1000)
    vects = np.vstack(subset.w2v.values)
    avg_distance = vects.dot(w2v).mean()
    
    return avg_distance

a = df.head(10000).w2v.apply(get_distance)

In [33]:
40 / 1000 * df.shape[0] / 3600

13.365244444444444

In [50]:
from multiprocessing import Pool

num_partitions = 10 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

parallelize_dataframe(df.head(100).w2v, get_distance)

ValueError: shapes (1000,32) and (10,) not aligned: 32 (dim 1) != 10 (dim 0)

In [46]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
mem_vars = sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)
print(sum(map(lambda x: x[1], mem_vars)) / 1000 / 1000, "mb")

582.228786 mb


582.236354